# Cuaderno 30: Parámetros para generación de planos cortantes

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

En este cuaderno revisitaremos la formulación MTZ del problema del agente viajero asimétrico para explorar cómo los parámetros de configuración del solver Gurobi pueden ser usados para mejorar el desempeño del método de branch-and-cut.

Dados: 
* un grafo dirigido **completo** $D=(V,A)$, con $V=\{1, \ldots, n\}$; y,
* un vector $c \in \ZZ^{A}$ de costos asociados a los arcos de $D$.

El *problema del agente viajero asimétrico (Asymmetric Traveling Salesman Problem, ATSP)* consiste en encontrar un circuito dirigido que visite **todos** los nodos de $D$ y que tenga el menor costo posible.

En el Cuaderno 16 presentamos la formulación compacta de [C.E.Miller, A.W.Tucker y R.A.Zemlin (MTZ, 1960)](https://dl.acm.org/doi/abs/10.1145/321043.321046?casa_token=wS9ir40FaVQAAAAA%3AREqLlEQWddvpSw0sEq2toPwlctyB4Tfa2O2aUHj3WqIfPlXESLWYFByDmy1UWEqcdIFZG6RibRhLXw) para el ATSP, la cual consiste en introducir variables auxiliares de ordenamiento $u_i$ para los nodos $i \in V \setminus \{ 1 \}$, y no requiere un número exponencial de restricciones:

\begin{align*}
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{(j, i) \in A} x_{ji} = 1, \quad \forall i \in V,\\
&\sum_{(i, j) \in A} x_{ij} = 1, \quad \forall i \in V,\\ 
& u_j \geq u_i + (1 + n) x_{ij} - n, \quad\forall (i,j) \in A, i \neq 1, j \neq 1, \\
& x_{ij} \in \{0, 1\}, \quad \forall (i, j) \in A,\\
& u_i \in \{2, \ldots, n \} \quad \forall i \in V \setminus \{1\}.\\
\end{align*}

La función objetivo mide el costo total de los arcos seleccionados en el tour.

Las dos primeras familias de restricciones corresponden a las restricciones de grado de los nodos.

La tercera familia de restricciones requiere que, para cualquier arco $(i,j)$ cuyos dos extremos sean distintos al nodo $1$, si el arco es seleccionado dentro de la solución, entonces debe cumplirse que $u_j \geq u_i + 1$. Notar que de esta manera se evitan soluciones que incluyan ciclos que no contienen al nodo 1. Como resultado, se eliminan soluciones con subtoures, empleando únicamente $m -2n +2$ restricciones, donde $m=n(n-1)$ es el número de arcos en el grafo.

Esta formulación tiene como ventaja que nos permite abordar instancias de mayor tamaño, con respecto a la formulación con desigualdades de eliminación de subciclos (sin emplear desigualdades tipo lazy). Sin embargo, su desventaja principal radica en el hecho de que las cotas duales son débiles, lo que empeora el desempeño del método de branch-and-bound. En este cuaderno, exploraremos alternativas de configuración de Gurobi para intensificar el uso de planos cortantes con el propósito de mejorar estas cotas.


Definimos primero los datos. Usaremos la función `randint` para generar puntos con coordenadas aleatorias en el rango {0,..,100}. El costo de un arco $(i,j)$ será igual a la distancia euclideana entre $i$ y $j$:

In [ ]:
from gurobipy import *
import random as rm
import math 

# Numero de nodos del grafo
n = 100

# Nodos del grafo
V = tuplelist(range(1,n+1))

# Nodos sin el primero
V2 = V[1:]

# Posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
 
A = c.keys()


Emplearemos el módulo `matplotlib` para graficar el tour de la solución. Definiremos para ello la función `dibujarTour` que recibe tres argumentos: una lista `coordx` con las coordenadas horizontales de los nodos, una lista `coordy` con las coordenadas verticales y un vector `tour` con una permutación de los nodos indicando el orden de visita en la solución.

In [ ]:
import matplotlib.pyplot as plt
import random

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)
    display(plt.show())


Definimos ahora el objeto modelo, las variables, la función objetivo y las restricciones. 

In [ ]:
# Objeto modelo
m = Model('atsp-compacto')

# Variables de selección de arcos
x = m.addVars(A, name="x", vtype=GRB.BINARY)

# Variables de ordenamiento de nodos
u = m.addVars(V2, name="u", vtype=GRB.INTEGER, lb=2, ub=n)

# Función objetivo
m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

# Restricciones de grado saliente
m.addConstrs((x.sum(i,'*')  == 1 for i in V), 
                 "g_saliente")
    
# Restricciones de grado entrante
m.addConstrs((x.sum('*', i)  == 1 for i in V), 
                 "g_entrante")

# Restricciones de ordenamiento de nodos
m.addConstrs((u[j]  >= u[i] + (1 + n)*x[i,j] - n 
              for i,j in A if i!=1 and j!=1), "ordenamiento");

Establecemos un tiempo límite para la solución del modelo y una tolerancia para la brecha de optimalidad:

In [ ]:
# Terminar al alcanzar un Gap del 1%
m.Params.MIPGap = 0.01

# Terminar luego de 180 segundos
m.Params.TimeLimit = 180

Finalmente, resolvemos el modelo.

In [ ]:
# Resolver el modelo
m.optimize()


## Parámetros de Gurobi

El comportamiento de los algoritmos de solución implementados en Gurobi puede ser controlado a través de distintos *parámetros*. Para asignar valores a estos parámetros existen distintos métodos equivalentes. En el API Python puede accederse, por ejemplo, al atributo `Params` del objeto modelo, tal como lo hemos hecho para fijar la toleracia para la brecha de optimalidad (parámetro [`MIPGap`](https://www.gurobi.com/documentation/10.0/refman/mipgap2.html)) y el tiempo límite para le ejecución del modelo (parámetro [`TimeLimit`](https://www.gurobi.com/documentation/10.0/refman/timelimit.html)).

Una [lista completa de todos los parámetros](https://www.gurobi.com/documentation/10.0/refman/parameter_descriptions.html) está disponible a través del manual de referencia del solver.

### MIPFocus

El parámetro [`MIPFocus`](https://www.gurobi.com/documentation/10.0/refman/mipfocus.html) controla estrategia general de solución del método de branch-and-bound, entre otras cosas a través del balance entre el tiempo dedicado a la generación de cortes y a las heurísticas primales. Por defecto, este parámetro tiene el valor de 0, y puede recibir valores enteros entre 0 y 3.

Asignando un valor de `MIPFocus = 3` se pide al solver enfocar su estrategia en la mejora de la cota dual a través de la generación de planos cortantes.

Observar que antes de volver a correr un modelo, debemos llamar al método `reset()` para descartar la información de la solución anterior.

In [ ]:
# Eliminar información de solución anterior
m.reset()

# Enfocar estrategia en mejoramiento de la cota dual
m.Params.MIPFocus = 3

# Resolver el modelo
m.optimize()

Por otra parte, si tenemos razones para suponer que una solución disponible es (casi) óptima, podemos asignar el valor de `MIPFocus = 2`, para pedir a Gurobi enfocar su estrategia en probar la optimalidad.

In [ ]:
# Descartar información de solución anterior
m.reset()

# Enfocar estrategia en probar optimalidad
m.Params.MIPFocus = 2

# Resolver el modelo
m.optimize()

Es posible combinar varias estrategias en el proceso de solución de una instancia. Por ejemplo, supongamos que queremos resolver la instancia anterior con una brecha de optimalidad del 0.1%. Conocemos que fijando `MIPFocus=3` podemos obtener una brecha inferior al 1% en 180s, así que empezamos con este valor: 

In [ ]:
# Fijar brecha de optimalidad en 0.1%
m.Params.MIPGap=0.001

# Enfocar estrategia en mejorar la cota dual
m.Params.MIPFocus = 3

# Descartar información de solución anterior
m.reset()

# Resolver el modelo
m.optimize()

Ahora disponemos de una solución bastante cercana al óptimo. Cambiamos la estrategia del solver a probar optimalidad, y continuamos la solución (por 180s más).

In [ ]:
# Cambiar a estrategia de probar optimalidad
m.Params.MIPFocus = 2

# Continuar solución del modelo
m.optimize()

Podemos graficar la solución encontrada:

In [ ]:
# Crear lista con arcos seleccionados en la solucion
vx = m.getAttr('x', x)
L = tuplelist([(i,j) for i,j in A if vx[i,j]>0.1])

# Recuperar el tour como un ordenamiento de los nodos
T = [1]
# nodo actual:
i = 1
while True:
    # Determinar sucesor de i
    a = L.select(i,'*')[0]
    L.remove(a)
    # Colocar sucesor en la lista del tour y actualizar i
    T.append(a[1])
    i = a[1]
    # Terminar cuando el nodo colocado sea 1
    if i==1: 
        break;
        
# Graficar el tour
dibujarTour(coordx, coordy, T)    

### Cuts

El parámetro [`Cuts`](https://www.gurobi.com/documentation/10.0/refman/cuts.html) controla directamente la intensidad de la generación de planos de cortantes durante el proceso de solución. Este parámetro puede tomar valores enteros entre -1 y 3, con un valor por defecto de -1.

Fijar `Cuts=0` desactiva la generación de planos cortantes. Fijar valores más altos hace que la generación sea más agresiva, es decir, que el solver intente agregar una mayor cantidad de planos cortantes.

In [ ]:
# Establecer nuevamente la brecha de tolerancia en 1%
m.Params.MIPGap=0.01

# Restablecer valor por defecto a MIPFocus
m.Params.MIPFocus = 0

# Fijar generación muy agresiva de cortes
m.Params.Cuts = 3

# Descartar información de la solución anterior
m.reset()

# Resolver el modelo
m.optimize()


También es posible controlar la intensidad de la generación de cada familia individual de cortes, a través de parámetros con nombres respectivos. Generalmente, estos parámetros enteros pueden recibir valores entre -1 y 2, con un valor por defecto de -1. Un valor de 0 desactiva la generación de cortes esa familia, un valor de 1 establece una generación de cortes moderada, y un valor de 2 una generación de cortes agresiva.

Por ejemplo, podemos fijar la generación agresiva de cortes solamente para las familias `FlowCoverCuts`, `ZeroHalfCuts` y `RelaxLiftCuts` que son aquellas en las que se encontraron más planos cortantes durante las soluciones anteriores.

In [ ]:
# Fijar la generación de cortes global en su valor por defecto
m.Params.Cuts = -1

# Fijar generación agresiva de cortes tipo FlowCoverCuts
m.Params.FlowCoverCuts = 2

# Fijar generación agresiva de cortes tipo ZeroHalfCuts
m.Params.ZeroHalfCuts = 2

# Fijar generación agresiva de cortes tipo RelaxLiftCuts
m.Params.RelaxLiftCuts = 2

# Descartar información de solución anterior
m.reset()

# Resolver el modelo
m.optimize()


### LogFile

A menudo es importante realizar varias corridas de un mismo modelo y guardar los registros de salida de Gurobi para poder compararlos luego. El parámetro [`LogFile`](https://www.gurobi.com/documentation/9.5/refman/logfile.html) permite especificar un nombre de archivo en donde se escribirá una copia de la salida de Gurobi.

Vamos a enviar a un archivo la salida correspondiente a la combinación de parámetros que produjo los mejores resultados en las pruebas.

In [ ]:
# Fijar generación de cortes tipo FlowCoverCuts al valor por defecto
m.Params.FlowCoverCuts = -1

# Fijar generación de cortes tipo ZeroHalfCuts al valor por defecto
m.Params.ZeroHalfCuts = -1

# Fijar generación de cortes tipo RelaxLiftCuts al valor por defecto
m.Params.RelaxLiftCuts = -1

# Fijar estrategia en mejoramiento de la cota dual
m.Params.MIPFocus = 3

# Archivo de salida para el registro de Gurobi
m.Params.LogFile = 'MTZ-ATSP-MIPFocus=3.log'

# Descartar información de soluciones anteriores
m.reset()

# Resolver el modelo
m.optimize()